# Clasificación Naive-Bayes

En cuanto al modelo de Naive-Bayes a usar, se ha elegido GaussianNB porque es la mejor opción para datos continuos que se distribuyen de forma normal, además de que es bastante robusto y suele ofrecer buenos resultados incluso en casos no ideales. Se impone a otros modelos como es el caso de:
- MultinomialNB ya que está diseñado para datos discretos
- BernoulliNB porque es específico para datos binarios (0 o 1), como presencia/ausencia de características.
- CategoricalNB ya que es específico para variables categóricas puras 
- ComplementNB por el mismo motivo que MultinomialNB

## Importación de librerías

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_predict

## Carga de datos

In [15]:
data_path = './data/'

train_data = pd.read_csv(f'{data_path}train_data.csv')
test_data = pd.read_csv(f'{data_path}test_data.csv')

df_reduce_mrmr = pd.read_csv(f'{data_path}X_train_reduce_mrmr.csv')
df_reduce_mrmr_instances = pd.read_csv(f'{data_path}df_reduce_mrmr_instances.csv')
df_reduce_mrmr_instances_hard = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_hard.csv')
df_reduce_mrmr_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_mrmr_instances_GLVQ.csv')

df_X_train_reduce_RFC = pd.read_csv(f'{data_path}df_X_train_reduce_RFC.csv')
df_reduce_RFC_instances = pd.read_csv(f'{data_path}df_reduce_RFC_instances.csv')
df_reduce_RFC_instances_hard = pd.read_csv(f'{data_path}df_reduce_RFC_instances_hard.csv')
df_reduce_RFC_instances_GLVQ = pd.read_csv(f'{data_path}df_reduce_RFC_instances_GLVQ.csv')

print("Datos cargados exitosamente:")
print(f"train_data: {train_data.shape}")
print(train_data["Class"].value_counts())
print(f"test_data: {test_data.shape}")
print(test_data["Class"].value_counts())
print(f"df_reduce_mrmr: {df_reduce_mrmr.shape}")
print(df_reduce_mrmr["Class"].value_counts())
print(f"df_reduce_mrmr_instances: {df_reduce_mrmr_instances.shape}")
print(df_reduce_mrmr_instances["Class"].value_counts())
print(f"df_reduce_mrmr_instances hard: {df_reduce_mrmr_instances_hard.shape}")
print(df_reduce_mrmr_instances_hard["Class"].value_counts())
print(f"df_reduce_mrmr_instances_GLVQ: {df_reduce_mrmr_instances_GLVQ.shape}")
print(df_reduce_mrmr_instances_GLVQ["Class"].value_counts())
print(f"df_X_train_reduce_RFC: {df_X_train_reduce_RFC.shape}")
print(df_X_train_reduce_RFC["Class"].value_counts())
print(f"df_reduce_RFC_instances: {df_reduce_RFC_instances.shape}")
print(df_reduce_RFC_instances["Class"].value_counts())
print(f"df_reduce_RFC_instances hard: {df_reduce_RFC_instances_hard.shape}")
print(df_reduce_RFC_instances_hard["Class"].value_counts())
print(f"df_reduce_RFC_instances_GLVQ: {df_reduce_RFC_instances_GLVQ.shape}")
print(df_reduce_RFC_instances_GLVQ["Class"].value_counts())

Datos cargados exitosamente:
train_data: (256326, 31)
Class
0    255883
1       443
Name: count, dtype: int64
test_data: (28481, 31)
Class
0    28432
1       49
Name: count, dtype: int64
df_reduce_mrmr: (256326, 11)
Class
0    255883
1       443
Name: count, dtype: int64
df_reduce_mrmr_instances: (886, 11)
Class
0    443
1    443
Name: count, dtype: int64
df_reduce_mrmr_instances hard: (886, 11)
Class
0    443
1    443
Name: count, dtype: int64
df_reduce_mrmr_instances_GLVQ: (2, 11)
Class
0    1
1    1
Name: count, dtype: int64
df_X_train_reduce_RFC: (256326, 11)
Class
0    255883
1       443
Name: count, dtype: int64
df_reduce_RFC_instances: (886, 11)
Class
0    443
1    443
Name: count, dtype: int64
df_reduce_RFC_instances hard: (886, 11)
Class
0    443
1    443
Name: count, dtype: int64
df_reduce_RFC_instances_GLVQ: (2, 11)
Class
0    1
1    1
Name: count, dtype: int64


## Función para entrenar y evaluar Naive Bayes

In [38]:
def train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    model = GaussianNB()
    y_pred_cv = cross_val_predict(model, X, y, cv=kfold)

    conf_matrix = confusion_matrix(y, y_pred_cv)
    report = classification_report(y, y_pred_cv, target_names=['Correctas', 'Fraudulentas'])

    print("Resultados de validación cruzada:")
    print("Matriz de confusión entrenamiento:")
    print(conf_matrix)
    print("\nReporte de Clasificación entrenamiento:")
    print(report)

    X_test_final = test_data[columns_to_keep]
    y_test_final = test_data['Class']

    scaler = MinMaxScaler()
    for col in ['Amount', 'Time']:
        if col in X_test_final.columns:
            X_test_final[col] = scaler.fit_transform(X_test_final[[col]])

    print(y_test_final.value_counts())

    model.fit(X, y)  
    y_pred_test = model.predict(X_test_final)

    conf_matrix_test = confusion_matrix(y_test_final, y_pred_test)
    report_test = classification_report(y_test_final, y_pred_test, target_names=['Correctas', 'Fraudulentas'])

    print("\nResultados en el conjunto de test:")
    print("Matriz de confusión del conjunto de test:")
    print(conf_matrix_test)
    print("\nReporte de Clasificación del conjunto de test:")
    print(report_test)

## Ejemplo con train data

In [39]:
X = train_data.drop(columns=['Class'])
y = train_data['Class']
columns = ["Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount"]

print("\n--- Evaluación con train data ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns)


--- Evaluación con train data ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[250284   5599]
 [    76    367]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99    255883
Fraudulentas       0.06      0.83      0.11       443

    accuracy                           0.98    256326
   macro avg       0.53      0.90      0.55    256326
weighted avg       1.00      0.98      0.99    256326

Class
0    28432
1       49
Name: count, dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])



Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[27718   714]
 [    8    41]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      0.97      0.99     28432
Fraudulentas       0.05      0.84      0.10        49

    accuracy                           0.97     28481
   macro avg       0.53      0.91      0.54     28481
weighted avg       1.00      0.97      0.99     28481



Con los datos de entrenamiento podemos observar como en la clase fraudulentas obtenemos una precision (0.05) y f1-score (0.10) muy bajas, debido a que GaussianNB es muy sensible al desbalanceo de las clases y favorece a la clase mayoritaria.

## Ejemplo con mRMR

In [40]:
X = df_reduce_mrmr.drop(columns=['Class'])
y = df_reduce_mrmr['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[251736   4147]
 [   136    307]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99    255883
Fraudulentas       0.07      0.69      0.13       443

    accuracy                           0.98    256326
   macro avg       0.53      0.84      0.56    256326
weighted avg       1.00      0.98      0.99    256326

Class
0    28432
1       49
Name: count, dtype: int64


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])



Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[27846   586]
 [   11    38]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      0.98      0.99     28432
Fraudulentas       0.06      0.78      0.11        49

    accuracy                           0.98     28481
   macro avg       0.53      0.88      0.55     28481
weighted avg       1.00      0.98      0.99     28481



Vemos que reduciendo las características con mRMR, aumentan muy ligeramente la precision, a 0.06 y el f1-score, a 0.11. La causa de unas métricas tan bajas, probablemente sea el desbalanceo de clases, tal y como se comentaba en el caso de los datos de entrenamiento. 

## Ejemplo con RFC

In [41]:
X = df_X_train_reduce_RFC.drop(columns=['Class'])
y = df_X_train_reduce_RFC['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[253712   2171]
 [    67    376]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      1.00    255883
Fraudulentas       0.15      0.85      0.25       443

    accuracy                           0.99    256326
   macro avg       0.57      0.92      0.62    256326
weighted avg       1.00      0.99      0.99    256326

Class
0    28432
1       49
Name: count, dtype: int64

Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[28151   281]
 [    6    43]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      0.99      0.99     28432
Fraudulentas       0.13      0.88      0.23        49

    accuracy                           0.99     28481
   macro avg       0.57      0.93      0.61     28481
weighted avg     

Al reducir las características usando RFC, vemos un aumento más considerable tanto en la precision (0.13) como en el f1-score (0.23) debido a que RFC hace un mejor manejo de los datos desbalanceados por su enfoque no lineal, que le permite capturar relaciones complejas, y por ser un conjunto de árboles, por lo que maneja mejor la variabilidad.

## Ejemplo con mRMR (ClusterCentroids_soft)

In [42]:
X = df_reduce_mrmr_instances.drop(columns=['Class'])
y = df_reduce_mrmr_instances['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_soft ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ClusterCentroids_soft ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[102 341]
 [ 15 428]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       0.87      0.23      0.36       443
Fraudulentas       0.56      0.97      0.71       443

    accuracy                           0.60       886
   macro avg       0.71      0.60      0.54       886
weighted avg       0.71      0.60      0.54       886

Class
0    28432
1       49
Name: count, dtype: int64

Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[ 1838 26594]
 [    1    48]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      0.06      0.12     28432
Fraudulentas       0.00      0.98      0.00        49

    accuracy                           0.07     28481
   macro avg       0.50      0.52      0.06     28481
weight

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Al reducir el número de instancias con mRMR, se ha reducido la precision y el f1-score para la clase minoritaria, hasta 0.00 en ambos casos. Sin embargo, en el conjunto de entrenamiento se lograba una precisión de 0.56 y un f1-score de 0.71. Esto probablemente se dé porque al balancear las clases se ha introducido ruido que favorece a la clase fraudulentas, lo que empeora el rendimiento del modelo.

## Ejemplo con RFC (ClusterCentroids_soft)

In [43]:
X = df_reduce_RFC_instances.drop(columns=['Class'])
y = df_reduce_RFC_instances['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_soft ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ClusterCentroids_soft ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[428  15]
 [ 82 361]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       0.84      0.97      0.90       443
Fraudulentas       0.96      0.81      0.88       443

    accuracy                           0.89       886
   macro avg       0.90      0.89      0.89       886
weighted avg       0.90      0.89      0.89       886

Class
0    28432
1       49
Name: count, dtype: int64

Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[28390    42]
 [    8    41]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.49      0.84      0.62        49

    accuracy                           1.00     28481
   macro avg       0.75      0.92      0.81     28481
weighte

En este caso, el balanceo de las clases y la reducción de instancias sí ha hecho que el rendimiento del modelo mejore bastante, logrando una precision de 0.49 y un f1-score de 0.62 para la clase fraudulentas.

## Ejemplo con mRMR (ClusterCentroids_hard)

In [44]:
X = df_reduce_mrmr_instances_hard.drop(columns=['Class'])
y = df_reduce_mrmr_instances_hard['Class']
columns_to_keep_mrmr = ['V17', 'Time', 'Amount', 'V25', 'V20', 'V7', 'V13', 'V22', 'V19', 'V23']

print("\n--- Evaluación con mRMR ClusterCentroids_hard ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR ClusterCentroids_hard ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[ 98 345]
 [ 17 426]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       0.85      0.22      0.35       443
Fraudulentas       0.55      0.96      0.70       443

    accuracy                           0.59       886
   macro avg       0.70      0.59      0.53       886
weighted avg       0.70      0.59      0.53       886

Class
0    28432
1       49
Name: count, dtype: int64

Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[ 1954 26478]
 [    1    48]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      0.07      0.13     28432
Fraudulentas       0.00      0.98      0.00        49

    accuracy                           0.07     28481
   macro avg       0.50      0.52      0.07     28481
weight

C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\3805750772.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[col] = scaler.fit_transform(X_test_final[[col]])


Los resultados son idénticos a los obtenidos con ClusterCentroids_soft

## Ejemplo con RFC (ClusterCentroids_hard)

In [45]:
X = df_reduce_RFC_instances_hard.drop(columns=['Class'])
y = df_reduce_RFC_instances_hard['Class']
columns_to_keep_RFC = ['V17', 'V16', 'V12', 'V14', 'V11', 'V10', 'V9', 'V4', 'V18', 'V7']

print("\n--- Evaluación con RFC ClusterCentroids_hard ---")
train_and_evaluate_naive_bayes_with_cv(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC ClusterCentroids_hard ---
Resultados de validación cruzada:
Matriz de confusión entrenamiento:
[[428  15]
 [ 82 361]]

Reporte de Clasificación entrenamiento:
              precision    recall  f1-score   support

   Correctas       0.84      0.97      0.90       443
Fraudulentas       0.96      0.81      0.88       443

    accuracy                           0.89       886
   macro avg       0.90      0.89      0.89       886
weighted avg       0.90      0.89      0.89       886

Class
0    28432
1       49
Name: count, dtype: int64

Resultados en el conjunto de test:
Matriz de confusión del conjunto de test:
[[28391    41]
 [    8    41]]

Reporte de Clasificación del conjunto de test:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.50      0.84      0.63        49

    accuracy                           1.00     28481
   macro avg       0.75      0.92      0.81     28481
weighte

Los resultados son ligeramente mejores a los obtenidos con ClusterCentroids_soft con una precisión de 0.50 y un f1-score de 0.63.

## Ejemplo con mRMR (GLVQ)

Para entrenar los modelos usando los datos que han reducido su número de instancias usando GLVQ, se define una nueva función que no use validación cruzada debido a que el dataset cuenta únicamente con dos instancias, que son los prototipos obtenidos.

In [46]:
def train_and_evaluate_naive_bayes(X, y, test_data, columns_to_keep):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Dimensiones de los conjuntos:")
    print(f"Conjunto de entrenamiento: {X_train.shape}, {y_train.shape}")
    print(f"Conjunto de prueba: {X_test.shape}, {y_test.shape}")

    model = GaussianNB()
    model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    print(f"Precisión en el conjunto de entrenamiento: {accuracy:.2f}")

    X_test_final = test_data[columns_to_keep]
    y_test_final = test_data['Class']

    scaler = MinMaxScaler()
    for col in ['Amount', 'Time']:
        if col in X_test_final.columns:
            X_test_final[col] = scaler.fit_transform(X_test_final[[col]])

    print(y_test_final.value_counts())

    y_pred = model.predict(X_test_final)

    conf_matrix = confusion_matrix(y_test_final, y_pred)
    report = classification_report(y_test_final, y_pred, target_names=['Correctas', 'Fraudulentas'])

    print("Matriz de confusión:")
    print(conf_matrix)
    print("\nReporte de Clasificación:")
    print(report)

In [47]:
X = df_reduce_mrmr_instances_GLVQ.drop(columns=['Class'])
y = df_reduce_mrmr_instances_GLVQ['Class']

print("\n--- Evaluación con mRMR GLVQ ---")
train_and_evaluate_naive_bayes(X, y, test_data, columns_to_keep_mrmr)


--- Evaluación con mRMR GLVQ ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (1, 10), (1,)
Conjunto de prueba: (1, 10), (1,)
Precisión en el conjunto de entrenamiento: 0.00
Class
0    28432
1       49
Name: count, dtype: int64
Matriz de confusión:
[[28432     0]
 [   49     0]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.00      0.00      0.00        49

    accuracy                           1.00     28481
   macro avg       0.50      0.50      0.50     28481
weighted avg       1.00      1.00      1.00     28481



c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:513: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:514: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:514: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_13428\1184420830.py:24: SettingWithCopyWarning: 
A 

Cuando se reducen las instancias con GLVQ se observa que nunca se predice la clase fraudulenta. Esto es debido a que GaussianNB depende de supuestos estadísticos como normalidad y varianza, por lo que al trabajar simplemente con un prototipo de cada clase, no es capaz de distinguir claramente las clases.

## Ejemplo con RFC (GLVQ)

In [7]:
X = df_reduce_RFC_instances_GLVQ.drop(columns=['Class'])
y = df_reduce_RFC_instances_GLVQ['Class']

print("\n--- Evaluación con RFC GLVQ ---")
train_and_evaluate_naive_bayes(X, y, test_data, columns_to_keep_RFC)


--- Evaluación con RFC GLVQ ---
Dimensiones de los conjuntos:
Conjunto de entrenamiento: (1, 10), (1,)
Conjunto de prueba: (1, 10), (1,)
Precisión en el conjunto de entrenamiento: 0.00
Class
0    28432
1       49
Name: count, dtype: int64
Matriz de confusión:
[[28432     0]
 [   49     0]]

Reporte de Clasificación:
              precision    recall  f1-score   support

   Correctas       1.00      1.00      1.00     28432
Fraudulentas       0.00      0.00      0.00        49

    accuracy                           1.00     28481
   macro avg       0.50      0.50      0.50     28481
weighted avg       1.00      1.00      1.00     28481



c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:513: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:514: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practica\dos\PracticaMineriaDatos\.venv\Lib\site-packages\sklearn\naive_bayes.py:514: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
c:\Users\Usuario\Desktop\Apuntes\Máster\Minería de datos-Preprocesamiento y Clasificacion\Practic

Ocurre lo mismo que con mRMR.

# Conclusiones

En resumen, este es el rendimiento obtenido con los modelos GaussianNB entrenados con diferentes conjuntos de datos:

| Datos                       | Precision fraudulentas | Recall fraudulentas | F1-score fraudulentas |
|-----------------------------|------------------------|---------------------|-----------------------|
| Dataset completo             | 0.05                   | 0.84                | 0.10                  |
| mRMR                         | 0.06                   | 0.78                | 0.11                  |
| RFC                          | 0.13                   | 0.88                | 0.23                  |
| mRMR+clusterCentroids        | 0.00                   | 0.98                | 0.00                  |
| RFC+clusterCentroids_soft    | 0.49                   | 0.84                | 0.62                  |
| RFC+clusterCentroids_hard    | 0.50                   | 0.84                | 0.63                  |
| mRMR+GLVQ                    | 0.00                   | 0.00                | 0.00                  |
| RFC+GLVQ                     | 0.00                   | 0.00                | 0.00                  |

Se observa que RFC es la técnica de reducción de características que mejor ha funcionado en este caso, además, al combinarla con la reducción de instancias con clusterCentroids con el método de hard de voto se ha logrado el mejor rendimiento con Naive-Bayes, seguido muy de cerca por clusterCentroids con el método soft de voto. El rendimiento es algo bajo, probablemente debido a la simplicidad de Naive-Bayes y las características del dataset.